A continuació anirem exposant els diferents passos que s’han seguit per generar aquest dataset. Adjunt al repositori del git hi ha un document pdf on hi ha ampliada la informació que pugui faltar en aquest notebook de jupyter per tal de que tots els passos de la implementació fets quedin clars. Es recomana llegir primer el pdf.

1.- Importem les llibreries necessàries per poder realitzar la pràctica.

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

2.- Definim la funció getInfoPais a la qual li passarem la url que volem analitzar i ens retornarà un objecte BeautifulSoup formatat.

In [2]:
def getInfoPais(url):
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

3.- Cridem la funció anterior, li passem la url que volem estudiar i busquem quines taules tenim. Identifiquem quina classe o quin id te la taula que conté el llistat de països que volem emmagatzemar al data set.

In [3]:
info = getInfoPais('https://datosmacro.expansion.com/deuda')
taules = info.find_all('table')
for taula in taules:
    print(taula.prettify())  

<table class="table tabledat table-striped table-condensed table-hover" id="tb1_318">
 <thead>
  <tr class="tableheader">
   <th>
    Países
   </th>
   <th>
    Fecha
   </th>
   <th class="eur" colspan="2" title="Deuda pública en Millones de EUR">
    Deuda total (M.€)
   </th>
   <th class="dol" colspan="2" title="Deuda pública en Millones de USD">
    Deuda total (M.$)
   </th>
   <th title="Deuda pública en porcentaje PIB">
    Deuda (%PIB)
   </th>
   <th class="eur" title="Deuda pública per cápita en EUR">
    Deuda Per Cápita
   </th>
   <th class="dol" title="Deuda pública per capita en USD">
    Deuda Per Cápita
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    <a href="/deuda/espana">
     España [+]
    </a>
   </td>
   <td class="fecha" data-value="2018-12-01">
    2018
   </td>
   <td class="numero eur" data-value="1173303">
    1.173.303
   </td>
   <td class="hbar eur wdsp1_2">
    <div class="graph_hbar" style="background-color: #F1BF00; width:6.5%;" title="1.173.

4.- En aquest cas la taula que volem analitzar podem veure que te per classe class="table tabledat table-striped table-condensed table-hover" i per identificador id="tb1_318". Utilitzarem per exemple, l'identificador id="tb1_318".

In [4]:
taula = info.find('table', {'id': 'tb1_318'})
print(taula.prettify())

<table class="table tabledat table-striped table-condensed table-hover" id="tb1_318">
 <thead>
  <tr class="tableheader">
   <th>
    Países
   </th>
   <th>
    Fecha
   </th>
   <th class="eur" colspan="2" title="Deuda pública en Millones de EUR">
    Deuda total (M.€)
   </th>
   <th class="dol" colspan="2" title="Deuda pública en Millones de USD">
    Deuda total (M.$)
   </th>
   <th title="Deuda pública en porcentaje PIB">
    Deuda (%PIB)
   </th>
   <th class="eur" title="Deuda pública per cápita en EUR">
    Deuda Per Cápita
   </th>
   <th class="dol" title="Deuda pública per capita en USD">
    Deuda Per Cápita
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    <a href="/deuda/espana">
     España [+]
    </a>
   </td>
   <td class="fecha" data-value="2018-12-01">
    2018
   </td>
   <td class="numero eur" data-value="1173303">
    1.173.303
   </td>
   <td class="hbar eur wdsp1_2">
    <div class="graph_hbar" style="background-color: #F1BF00; width:6.5%;" title="1.173.

5.- Coses a tenir en compte:

La taula que volem tractar podem veure que conté una llista de països i per cada fila, tenim el nom del país, el deute total, el deute en funció del PIB, el deute per càpita i la data (anualitat) en la qual es basen les dades del deute.

La primera columna de la taula a part del nom del país, conté l’enllaç per anar a la pàgina que conté informació ampliada del país en qüestió.

És a dir, estem analitzant la taula de la pàgina:

https://datosmacro.expansion.com/deuda

Si fem clic a un país d'aquesta taula ens porta a la pàgina:

https://datosmacro.expansion.com/deuda/espana

La qual amplia la informació sobre el deute del país en qüestió. Però en aquest cas el que volem es accedir a la pàgina que amplia la informació genèrica del país, que en aquest cas seria:

https://datosmacro.expansion.com/paises/espana

L'objectiu és extreure uns altres indicador i mirar d’establir relacions entre aquests indicadors i el deute del país.

Per tant, recorrerem la llista, agafarem l’enllaç, substituirem 'deuda' per 'paises' i accedirem a la pàgina de cada país per extreure informació extra que afegirem al nostre data set.

6.- Prova per assegurar-nos que som capaços de generar tots els enllaços que necessitarem a posteriori:

In [5]:
files = taula.find_all('tr')

for fila in files:
    celes = fila.find_all('td')
    if len(celes) > 1:
        urlExtra = celes[0].find('a').get('href')        
        urlExtra = urlExtra.replace('deuda','paises')
        print(urlExtra)
    


/paises/espana
/paises/alemania
/paises/uk
/paises/francia
/paises/italia
/paises/portugal
/paises/usa
/paises/japon
/paises/china
/paises/emiratos-arabes-unidos
/paises/afganistan
/paises/antigua-barbuda
/paises/albania
/paises/armenia
/paises/angola
/paises/argentina
/paises/austria
/paises/australia
/paises/azerbaiyan
/paises/bosnia-herzegovina
/paises/barbados
/paises/banglades
/paises/belgica
/paises/burkina-faso
/paises/bulgaria
/paises/barein
/paises/burundi
/paises/benin
/paises/brunei
/paises/bolivia
/paises/brasil
/paises/bahamas
/paises/butan
/paises/botsuana
/paises/bielorrusia
/paises/belice
/paises/canada
/paises/republica-democratica-congo
/paises/republica-centroafricana
/paises/congo
/paises/suiza
/paises/costa-marfil
/paises/chile
/paises/camerun
/paises/colombia
/paises/costa-rica
/paises/cabo-verde
/paises/chipre
/paises/republica-checa
/paises/yibuti
/paises/dinamarca
/paises/dominica
/paises/republica-dominicana
/paises/argelia
/paises/ecuador
/paises/estonia
/pai

7.- Implementem una funció auxiliar a la qual li passarem la url ‘extra’ que hem comentat a l’apartat anterior i farem el tractament per tal de retornar aquests camps extra.
En aquest cas, el que fa aquesta funció és:

-	Rep la url, per exemple ‘https://datosmacro.expansion.com/paises/espana’
-	Identifica la taula que conte la informació que ens interessa
-	Extreu els camps d’interès següents:

    -- Índex de corrupció, Índex de fragilitat, Taxa d’atur, Aturats, Salari mínim, Salari mig, Població, Immigrants, Emigrants, Assassinats, Emissions CO2 per càpita
    

-	Retorna la informació, controlant que si hi ha països que no tenen informació d’algun camp retorna el camp buit


In [6]:
def getInfoExtraPais(url):
    
        infoPais = getInfoPais('https://datosmacro.expansion.com' + url)
        taula = infoPais.find('table', {'class': 'table tabledat table-striped table-condensed table-hover'})
        extraInfo = []        
        
        infoCorrupcio = []
        infoFragilitat = []
        infoTaxaAtur = []
        infoAturats = []
        infoSalariMin = []
        infoSalariMig = []
        infoPoblacio = []
        infoImmigrants = []
        infoEmmigrants = []
        infoAssessinats = []
        infoEspeVida = []
        infoCO2 = []
        
        files = taula.find_all('tr')
        
        for fila in files:
            
            celes = fila.find_all('td')
            
            if (celes[0].find('a', {'title': 'Índice de Percepción de la Corrupción'}) != None):
                infoCorrupcio.append(celes[1].text)
                infoCorrupcio.append(celes[2].text)    

            if (celes[0].find('a', {'title': 'Índice Mundial de Fragilidad'}) != None):
                infoFragilitat.append(celes[1].text)
                infoFragilitat.append(celes[2].text)
                
            if (celes[0].find('a', {'title': 'Tasa de desempleo'}) != None):
                infoTaxaAtur.append(celes[1].text)
                infoTaxaAtur.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Parados'}) != None):
                infoAturats.append(celes[1].text)
                infoAturats.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Salario Mínimo '}) != None):                
                infoSalariMin.append(celes[1].text)
                infoSalariMin.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Salario Medio'}) != None):                
                if "$" not in celes[2].text:     
                    infoSalariMig.append(celes[1].text)
                    infoSalariMig.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Población'}) != None):
                infoPoblacio.append(celes[1].text)
                infoPoblacio.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Inmigrantes'}) != None):
                infoImmigrants.append(celes[1].text)
                infoImmigrants.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Emigrantes totales'}) != None):
                infoEmmigrants.append(celes[1].text)
                infoEmmigrants.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Homicidios Intencionados'}) != None):
                infoAssessinats.append(celes[1].text)
                infoAssessinats.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Esperanza de vida al nacer'}) != None):
                infoEspeVida.append(celes[1].text)
                infoEspeVida.append(celes[2].text) 
                
            if (celes[0].find('a', {'title': 'Emisiones CO2 toneladas per capita'}) != None):
                infoCO2.append(celes[1].text)
                infoCO2.append(celes[2].text)             
        
        if (len(infoCorrupcio) > 1):
            extraInfo.append(infoCorrupcio[0])
            extraInfo.append(infoCorrupcio[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoFragilitat) > 1):
            extraInfo.append(infoFragilitat[0])
            extraInfo.append(infoFragilitat[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoTaxaAtur) > 1):
            extraInfo.append(infoTaxaAtur[0])
            extraInfo.append(infoTaxaAtur[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoAturats) > 1):
            extraInfo.append(infoAturats[0])
            extraInfo.append(infoAturats[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoSalariMin) > 1):
            extraInfo.append(infoSalariMin[0])
            extraInfo.append(infoSalariMin[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoSalariMig) > 1):
            extraInfo.append(infoSalariMig[0])
            extraInfo.append(infoSalariMig[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoPoblacio) > 1):
            extraInfo.append(infoPoblacio[0])
            extraInfo.append(infoPoblacio[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoImmigrants) > 1):
            extraInfo.append(infoImmigrants[0])
            extraInfo.append(infoImmigrants[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoEmmigrants) > 1):
            extraInfo.append(infoEmmigrants[0])
            extraInfo.append(infoEmmigrants[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoAssessinats) > 1):
            extraInfo.append(infoAssessinats[0])
            extraInfo.append(infoAssessinats[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')

        if (len(infoEspeVida) > 1):
            extraInfo.append(infoEspeVida[0])
            extraInfo.append(infoEspeVida[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')
            
        if (len(infoCO2) > 1):
            extraInfo.append(infoCO2[0])
            extraInfo.append(infoCO2[1])
        else: 
            extraInfo.append('')
            extraInfo.append('')

        return extraInfo

8.- Validem que la funció anterior funciona correctament. S’ha validat per països que tenen tota la informació i països que no tenen informats tots els camps, per exemple, Xina. 
Si no fèiem aquest tractament el dataset es construïa malament.

També s’ha fet una validació al camp que retorna el salari mig ja que aquest camp te el mateix identificador com a títol ‘Salario Medio’ però el retorna dos cops, un amb valor en € i un amb valor en $. D’aquesta manera, afegim un condicional per quedar-nos amb el valor en euros ja que tots els imports del dataset els gestionem amb aquesta unitat.


In [7]:
getInfoExtraPais('/paises/china')

['2018',
 '39',
 '2018',
 '72,4',
 '',
 '',
 '',
 '',
 '2018',
 '271,6 €',
 '',
 '',
 '2018',
 '1.395.380.000',
 '2017',
 '999.527',
 '2017',
 '9.962.058',
 '2016',
 '8.634',
 '2017',
 '76,41',
 '2017',
 '7,72']

9.- Generació del dataset

-	Guardem a la variable 'files' totes les files de la taula

Per cada fila fem:

-	Guardem a la variable 'celes' les diferents columnes de la fila

-	Les formatem i les guardem a la taula 'infoPaisos'

-	Agafem el valor de la primera cel·la que conté l’enllaç, li canviem ‘deuda’ per ‘paises’ i li passem la nova url a la funció que ens retornarà informació extra del país determinat

-	Afegim la informació extra a 'infoPaisos'

Quan finalitzem el bucle, és a dir, arribem al final de la llista de països, generem el dataset:

-	Eliminem 4 columnes que no ens interessen ja que la llista te algunes columnes que mostren un gràfic o mostren un import amb dòlars i nosaltres ens quedem les dades amb euros.

-	Agafem els headers de la taula principal, eliminem els que no ens interessen i posteriorment, manualment, assignem els headers que correspondran als camps extra de cada país.


In [8]:
infoPaisos = []
files = taula.find_all('tr')

for fila in files:
    celes = fila.find_all('td')
    if len(celes) > 1:
        infoPais = [cela.text.strip('\n') for cela in celes]
        
        urlExtra = celes[0].find('a').get('href')        
        urlExtra = urlExtra.replace('deuda','paises')
        
        extraInfo = getInfoExtraPais(urlExtra)
        
        infoPais += extraInfo
        print(infoPais)
        
        infoPaisos.append(infoPais)
        
dataset = pd.DataFrame(infoPaisos)

dataset.drop(dataset.columns[[3,4,5,8]], axis = 1, inplace = True) 

headers = files[0].find_all('th')
headers = [header.get_text().strip('\n') for header in headers]

del headers[3]
del headers[5]

headers += ['Anualidad índice corrupción']
headers += ['Índice de corrupción']

headers += ['Anualidad índice fragilidad']
headers += ['Índice mundial de fragilidad']

headers += ['Fecha tasa desempleo']
headers += ['Tasa de desempleo']

headers += ['Fecha parados']
headers += ['Parados']

headers += ['Anualidad salario mínimo']
headers += ['Salario mínimo']

headers += ['Anualidad salario medio']
headers += ['Salario medio']

headers += ['Anualidad población']
headers += ['Población']

headers += ['Anualidad inmigrantes']
headers += ['Inmigrantes']

headers += ['Anualidad emigrantes']
headers += ['Emigrantes']

headers += ['Anualidad homicidios intencionados']
headers += ['Homicidios intencionados']

headers += ['Anualidad esperanza de vida']
headers += ['Esperanza de vida']

headers += ['Anualidad emisiones CO2 toneladas per capita']
headers += ['Emisiones CO2 toneladas per capita']

dataset.columns = headers

dataset.to_csv("francesc_albuera_prac1.csv", index = False)


['España [+]', '2018', '1.173.303', '', '1.385.671', '', '97,60%', '24.999€', '29.523$', '2018', '58', '2018', '41,4', 'Septiembre 2019', '14,2%', 'III Trim 2019', '3.214 m.', '2019', '1.050,0 €', '2018', '26.923€', '2018', '46.934.632', '2017', '5.947.106', '2017', '1.345.862', '2016', '294', '2017', '83,40', '2017', '6,09']
['Alemania [+]', '2018', '2.063.172', '', '2.436.606', '', '60,90%', '24.852€', '29.350$', '2018', '80', '2018', '25,8', 'Septiembre 2019', '3,1%', 'II Trim 2019', '1.331 m.', '2019', '1.557,0 €', '2018', '50.546€', '2018', '83.019.214', '2017', '12.165.083', '2017', '4.208.083', '2016', '963', '2017', '81,10', '2017', '9,70']
['Reino Unido [+]', '2018', '2.054.185', '', '2.425.992', '', '86,80%', '30.822€', '36.401$', '2018', '80', '2018', '34,3', 'Agosto 2019', '3,9%', 'II Trim 2019', '1.252 m.', '2019', '1.524,5 €', '2018', '44.453€', '2018', '66.647.112', '2017', '8.841.717', '2017', '4.921.309', '2016', '791', '2017', '81,30', '2017', '5,73']
['Francia [+]', 

['Bolivia [+]', '2017', '17.150', '', '19.374', '', '51,26%', '1.532€', '1.731$', '2018', '29', '2018', '75,2', '', '', 'IV Trim 2017', '173 m.', '2017', '278,3 €', '', '', '2018', '11.353.142', '2017', '148.837', '2017', '820.722', '2016', '686', '2017', '69,47', '2017', '1,85']
['Brasil [+]', '2018', '1.389.520', '', '1.641.023', '', '87,89%', '6.634€', '7.834$', '2018', '35', '2018', '68,7', 'Agosto 2019', '11,8%', 'IV Trim 2018', '12.193 m.', '2018', '240,1 €', '', '', '2018', '209.469.333', '2017', '735.557', '2017', '1.612.860', '2016', '61.283', '2017', '75,72', '2017', '2,35']
['Bahamas [+]', '2018', '6.579', '', '7.773', '', '63,26%', '17.451€', '20.618$', '2018', '65', '2018', '50,0', '', '', '', '', '2018', '175,1 €', '', '', '2018', '377.000', '2017', '61.806', '2017', '41.684', '2016', '111', '2017', '75,82', '2017', '7,58']
['Bután [+]', '2018', '2.237', '', '2.642', '', '102,38%', '2.966€', '3.503$', '2018', '68', '2018', '74,3', '', '', '', '', '', '', '', '', '2018', '

['Finlandia [+]', '2018', '137.545', '', '162.441', '', '58,90%', '24.927€', '29.439$', '2018', '85', '2018', '17,9', 'Septiembre 2019', '6,7%', 'II Trim 2019', '215 m.', '', '', '2018', '43.984€', '2018', '5.517.919', '2017', '343.582', '2017', '294.631', '2016', '78', '2017', '81,70', '2017', '8,48']
['Fiyi [+]', '2018', '2.158', '', '2.548', '', '46,16%', '2.442€', '2.885$', '2005', '40', '2018', '74,5', '', '', '', '', '', '', '', '', '2018', '883.483', '2017', '13.911', '2017', '215.120', '2014', '20', '2017', '70,42', '2017', '1,59']
['Estados Federados de Micronesia [+]', '2018', '63', '', '75', '', '20,28%', '622€', '735$', '', '', '2018', '74,4', '', '', '', '', '', '', '', '', '2018', '102.000', '2017', '2.785', '2017', '20.754', '2015', '5', '2017', '69,32', '', '']
['Gabón [+]', '2018', '8.664', '', '10.232', '', '60,66%', '4.088€', '4.828$', '2018', '31', '2018', '72,5', '', '', '', '', '', '', '', '', '2018', '2.119.275', '2017', '280.197', '2017', '66.898', '2015', '155'

['Kiribati [+]', '2017', '35', '', '40', '', '21,62%', '313€', '354$', '2011', '31', '', '', '', '', '', '', '', '', '', '', '2018', '115.847', '2017', '3.022', '2017', '4.903', '2012', '8', '2017', '66,51', '2017', '0,26']
['Comoras [+]', '2018', '210', '', '248', '', '20,97%', '253€', '298$', '2018', '27', '2018', '82,6', '', '', '', '', '', '', '', '', '2018', '832.322', '2017', '12.555', '2017', '116.574', '2015', '60', '2017', '63,91', '2017', '0,25']
['San Cristóbal y Nieves [+]', '2018', '502', '', '593', '', '60,50%', '9.570€', '11.303$', '', '', '', '', '', '', '', '', '', '', '', '', '2018', '52.441', '2017', '7.587', '2017', '38.264', '2012', '18', '2002', '71,34', '2017', '4,30']
['Corea del Sur [+]', '2017', '541.803', '', '612.075', '', '37,68%', '10.536€', '11.902$', '2018', '57', '2018', '35,7', '', '', 'IV Trim 2017', '895 m.', '2018', '1.229,9 €', '2018', '37.078€', '2018', '51.635.000', '2017', '1.151.865', '2017', '2.477.575', '2016', '356', '2017', '82,63', '2017',

['Mozambique [+]', '2018', '12.165', '', '14.367', '', '99,84%', '412€', '487$', '2018', '23', '2018', '88,7', '', '', '', '', '', '', '', '', '2018', '29.495.962', '2017', '246.954', '2017', '653.251', '2011', '849', '2017', '58,87', '2017', '0,26']
['Namibia [+]', '2018', '5.685', '', '6.713', '', '45,79%', '2.355€', '2.781$', '2018', '53', '2018', '68,8', '', '', '', '', '', '', '', '', '2018', '2.414.000', '2017', '95.067', '2017', '190.132', '2012', '388', '2017', '64,85', '2017', '1,70']
['Níger [+]', '2017', '3.917', '', '4.425', '', '54,37%', '181€', '205$', '2018', '34', '2018', '96,2', '', '', '', '', '', '', '', '', '2018', '22.442.948', '2017', '295.610', '2017', '362.955', '2012', '788', '2017', '60,42', '2017', '0,12']
['Nigeria [+]', '2018', '91.873', '', '108.502', '', '27,26%', '469€', '554$', '2018', '27', '2018', '99,9', '', '', 'IV Trim 2016', '5.821 m.', '2013', '87,9 €', '', '', '2018', '195.874.740', '2017', '1.235.088', '2017', '1.255.425', '2015', '17.843', '20

['Eslovaquia [+]', '2018', '44.145', '', '52.135', '', '48,90%', '8.099€', '9.565$', '2018', '50', '2018', '42,5', 'Septiembre 2019', '5,6%', 'II Trim 2019', '155 m.', '2019', '520,0 €', '2018', '12.131€', '2018', '5.450.421', '2017', '184.642', '2017', '356.310', '2016', '57', '2017', '77,30', '2017', '6,95']
['Sierra Leona [+]', '2018', '2.176', '', '2.570', '', '62,99%', '285€', '336$', '2018', '30', '2018', '89,1', '', '', '', '', '', '', '', '', '2018', '7.650.154', '2017', '95.248', '2017', '159.017', '2015', '124', '2017', '52,21', '2017', '0,17']
['San Marino [+]', '2017', '1.037', '', '1.171', '', '76,64%', '31.134€', '35.172$', '', '', '', '', '', '', 'I Trim 2015', '2 m.', '', '', '', '', '2018', '34.590', '2017', '5.243', '2017', '2.342', '2011', '0', '2012', '85,60', '', '']
['Senegal [+]', '2018', '12.242', '', '14.457', '', '61,55%', '772€', '912$', '2018', '45', '2018', '79,6', '', '', 'IV Trim 2016', '154 m.', '', '', '', '', '2018', '15.854.360', '2017', '265.601', '2

['Sudáfrica [+]', '2018', '176.700', '', '208.683', '', '56,71%', '3.050€', '3.602$', '2018', '43', '2018', '72,9', 'Septiembre 2019', '29,1%', 'IV Trim 2018', '6.151 m.', '2013', '221,4 €', '', '', '2018', '57.939.000', '2017', '4.036.696', '2017', '898.407', '2016', '19.016', '2017', '63,40', '2017', '8,25']
['Zambia [+]', '2017', '14.168', '', '16.005', '', '61,85%', '841€', '950$', '2018', '35', '2018', '87,2', '', '', 'IV Trim 2017', '448 m.', '', '', '', '', '2018', '17.351.822', '2017', '156.982', '2017', '275.089', '2015', '853', '2017', '62,26', '2017', '0,29']
['Zimbabue [+]', '2017', '10.247', '', '14.505', '', '52,87%', '720€', '1.019$', '2018', '22', '2018', '102,3', '', '', '', '', '', '', '', '', '2018', '14.439.018', '2017', '403.866', '2017', '1.025.204', '2012', '981', '2017', '61,71', '2017', '0,73']
